In [22]:
from data_treat.get_ready_data import get_data
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import roc_curve, precision_recall_curve, auc # метрики качества
from sklearn.metrics import confusion_matrix, accuracy_score # метрики качества
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

In [2]:
data = get_data()
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,2,2,3,1,3,1,1,7,1,...,2,999,0,2,-1.8,92.893,-46.2,1.313,5099.1,1
1,39,8,3,4,1,1,1,2,7,1,...,4,999,0,2,1.1,93.994,-36.4,4.855,5191.0,1
2,25,8,2,4,1,3,1,2,5,5,...,1,999,0,2,1.4,94.465,-41.8,4.962,5228.1,1
3,38,8,2,3,1,2,2,2,5,1,...,3,999,0,2,1.4,94.465,-41.8,4.959,5228.1,1
4,47,1,2,7,1,3,1,1,8,2,...,1,999,0,2,-0.1,93.200,-42.0,4.191,5195.8,1


In [3]:
target_variable_name = 'y'
labels = data[target_variable_name]
data = data.drop(target_variable_name, axis=1)


In [4]:
labels.head()


0    1
1    1
2    1
3    1
4    1
Name: y, dtype: int32

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33)

In [6]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [7]:
from sklearn.neighbors import KNeighborsClassifier

In [8]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [13]:
 pd.value_counts(predictions)

1    1226
2     134
dtype: int64

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [15]:
accuracy_score(y_test, predictions)

0.8808823529411764

In [16]:
confusion_matrix(y_test, predictions)

array([[1130,   66],
       [  96,   68]], dtype=int64)

In [17]:
test_probabilities = model.predict_proba(x_test)
test_probabilities = test_probabilities[:, 1]

In [18]:
test_probabilities[:30]

array([0.        , 0.        , 1.        , 0.33333333, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.33333333, 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.33333333, 0.        ,
       0.        , 0.66666667, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [20]:
pr_auc_value = average_precision_score(y_test, test_probabilities)

In [21]:
pr_auc_value

0.8365043601942825

In [24]:
false_positive_rates, true_positive_rates, threshold = roc_curve(y_test, test_probabilities)

ValueError: y_true takes value in {1, 2} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.

In [23]:
plt.figure(figsize=(7, 7))

# рисуем кривую
plt.plot(false_positive_rates, true_positive_rates, label='Сглаженные значения ROC-AUC')

# кривая, соответствующая случайному угадыванию
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle=':', label='Модель, выдающая случайное значение')

plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.fill_between(false_positive_rates, true_positive_rates, step="mid", alpha=0.4, label='площадь под кривой (ROC-AUC)')
plt.legend()
plt.show()

NameError: name 'false_positive_rates' is not defined

<Figure size 504x504 with 0 Axes>